In [2]:
import pandas as pd
import numpy as np


In [5]:
df=pd.read_csv("/content/Sample - Superstore.csv", encoding='latin1')
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [14]:
a=df.shape


In [15]:
b=df.dtypes

In [16]:
c=df.isnull().mean() * 100

In [17]:
d=df.duplicated().sum()

In [19]:
def iqr_outliers(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return series[(series < lower) | (series > upper)]

outliers_sales = iqr_outliers(df["Sales"])
outliers_profit = iqr_outliers(df["Profit"])

In [12]:
corr_discount_profit = df["Discount"].corr(df["Profit"])
corr_sales_quantity = df["Sales"].corr(df["Quantity"])

In [20]:
with open("profiling_report.txt", "w") as f:
    f.write("==== DATA PROFILING REPORT ====\n\n")

    f.write(f"1. Number of rows & columns: {a}\n")

    f.write("2. Column Names & Data Types:\n")
    f.write(str(b) + "\n\n")

    f.write("3. Missing Value Percentage per Column:\n")
    f.write(str(c) + "\n\n")

    f.write(f"4. Duplicate Records Count: {d}\n\n")

    f.write("5. Outliers (IQR Method):\n")
    f.write(f"   Sales outliers: {outliers_sales}\n")
    f.write(f"   Profit outliers: {outliers_profit}\n\n")

    f.write("6. Relationship Checks (Correlation):\n")
    f.write(f"   Discount vs Profit: {corr_discount_profit:.4f}\n")
    f.write(f"   Sales vs Quantity: {corr_sales_quantity:.4f}\n\n")

print("profiling_report.txt generated successfully.")

profiling_report.txt generated successfully.


In [21]:
df = df.drop_duplicates()

In [22]:
date_cols = ["Order Date", "Ship Date"]
for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors="coerce")

In [23]:
for col in df.columns:
    if df[col].dtype in ["float64", "int64"]:
        df[col] = df[col].fillna(df[col].median())
    else:
        df[col] = df[col].fillna(df[col].mode()[0])

In [24]:
cat_cols = ["Category", "Sub-Category", "Segment"]

for col in cat_cols:
    df[col] = df[col].astype(str).str.strip().str.title()

# Example corrections (if dataset contains mistakes)
df["Segment"] = df["Segment"].replace({
    "Consumer ": "Consumer",
    "Home Office": "Home Office",
    "Corporate ": "Corporate"
})

In [25]:
numeric_cols = ["Sales", "Profit", "Quantity", "Discount"]

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# Fill any new NaNs after conversion
df[numeric_cols] = df[numeric_cols].fillna(0)

In [26]:
df.to_csv("cleaned_superstore.csv", index=False)

In [28]:
dictionary = pd.DataFrame({
    "Field Name": df.columns,
    "Data Type": [str(df[col].dtype) for col in df.columns],
    "Description": ["Description to be added" for _ in df.columns],
    "Cleaning Steps Applied": [
        "Duplicate removal, date parsing, missing value handling"
        for _ in df.columns
    ]
})